# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.


## Make imports and create the agent and the environment

In [1]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="./Tennis.app")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from agent import AgentList

agents = AgentList(state_size= 24, action_size=2, agents_count = 2, random_seed = 0)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


## Train the agents

In [ ]:

def ddpg(n_episodes: int =  600, max_actions:int = 1500 ):
    scores_0 = []
    scores_window_0 = deque(maxlen=100)
    scores_1 = []
    scores_window_1 = deque(maxlen=100)
    
    
    for i_episode in range(1,n_episodes+1):
        
        #reset the environment for every episode and initialize the state
        #step_brain_info = env.reset(train_mode=i_episode % 100 != 0)[brain_name] # reset the environment
        step_brain_info = env.reset(train_mode=True)[brain_name] # reset the environment
        
        agents.reset_noise()
        
        states = step_brain_info.vector_observations   #obtain the starting state
        
        score_0 = 0 #initialze the score for the episode
        score_1 = 0 #initialze the score for the episode
        
        
        for a in range(max_actions):
            
            #The agent chooses an action
            # after 200 episodes the actor should not add noise to the action to create a more stable agent
            actions = agents.act(states, add_noise=False)
            
            step_brain_info = env.step(actions)[brain_name]
            
            next_states: np.ndarray = step_brain_info.vector_observations   # get the next state
            rewards = step_brain_info.rewards                   # get the reward
            dones = step_brain_info.local_done                  # see if episode has finished
            
            score_0 += rewards[0]
            score_1 += rewards[1]
            
            #Take a step and learn from the s,a,r,s,d pair
            agents.step(states, actions, rewards, next_states, dones)
             
            states = next_states                             # roll over the state to next time step
            if agents.is_all_done(dones):                             # exit loop if episode finished
                break
                
            if env.global_done:
                print('Global Done Steps: '+ str(a))
                break
        
        scores_0.append(score_0)
        scores_1.append(score_1)
        scores_window_0.append(scores_0)       # save most recent score
        scores_window_1.append(scores_1)
        
        print('\rEpisode {}\tavg Score 0: {:.2f}, Score 1: {:.2f}'.format(i_episode, np.mean(scores_window_0), np.mean(scores_window_1)), end="")
        
        if i_episode % 100 == 0:
            print('\rEpisode {}\tavg Score 0: {:.2f}, Score 1: {:.2f}'.format(i_episode, np.mean(scores_window_0), np.mean(scores_window_1)))
        #if np.mean(scores_window)>=30.0:
        #    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
         #   #torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
        #    break
            
    
    return scores_0, scores_1
            
    
scores_0, scores_1 =  ddpg()
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores_0)), scores_0)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show() 

In [7]:
# Save the weights
agents.save_state()

# See the trained agent in action

In [8]:
# restore weights

agents.restore_state()

n_episodes = 2
for i_episode in range(1,n_episodes+1):
    step_brain_info = env.reset(train_mode=False)[brain_name] # reset the environment
    states = step_brain_info.vector_observations   #obtain the starting state
    
    for a in range(1000):
        
        #The agent chooses an action
        # after 200 episodes the actor should not add noise to the action to create a more stable agent
        actions = agents.act(states, add_noise=False)
            
        step_brain_info = env.step(actions)[brain_name]
        
        dones = step_brain_info.local_done
        
        if agents.is_all_done(dones):                             # exit loop if episode finished
            break
                
        if env.global_done:
            print('Global Done Steps: '+ str(a))
            break
        

FileNotFoundError: [Errno 2] No such file or directory: './weights/critic'

In [ ]:
env.close()